# Step-by-step Guide: Running Cellmaps VNN

The Cell Maps VNN Tool allows to create an interpretable neural network-based model that predicts cell response to a drug. The tool uses hierarchy in HCX format to create visible neural network that incorporates given input features. Then the tools allows training the model and perform predictions of cell response to a drug.

The tool creates an output directory (for both training and prediction) where results are stored and registered within Research Object Crates (RO-Crate) using the FAIRSCAPE-cli.

### Goals of VNN
- Predicts cell response to a drug
- Explainable data-driven method
- Constructs the neural network to mirror the hierarchical organization of a cell

### Overview of VNN Workflow

<img src="../docs/images/cellmaps_vnn_general.png" alt="VNN Workflow Overview" style="width:70%;">

### Installation

It is highly recommended to create conda virtual environment and run jupyter from there.

`conda create -n vnn_env python=3.8`

`conda activate vnn_env`

To install Cellmaps Pipeline run:

`pip install cellmaps_vnn`

### Drug response data

You should have your two separate sets of training and test data. Later, during the training the data is split into training and validation data.

### Cell Feature Data (required for both training and prediction steps)

Genetic alteration data: a panel of 718 clinical genes was assembled from the union of genes assessed by FoundationOne CDx, Tempus xT, PALOMA-3 trial or Project GENIE, each of which assesses mutations and/or copy number aberrations. To compile genotypes for all cell lines, non-synonymous coding mutations and copy number alterations were extracted for the 718 clinical panel genes from the Cancer Cell Line Encyclopedia. ([Park et al. Nat Cancer (2024)](https://doi.org/10.1038/s43018-024-00740-1))

- `gene2ind.txt`:
  
    A tab-delimited file where the 1st column is index of genes and the 2nd column is the name of genes.

    ```
    0       ABCB1
    1       ABCC3
    2       ABL1
    ```
    <br/>
- `cell2ind.txt`:
  
    A tab-delimited file where the 1st column is index of cells and the 2nd column is the name of cells (genotypes).

    ```
    0       201T_LUNG
    1       22RV1_PROSTATE
    2       2313287_STOMACH
    ```
    <br/>
- `cell2mutation.txt`:
    A comma-delimited file where each row has 718 binary values indicating each gene is mutated (1) or not (0). The column index of each gene should match with those in gene2ind.txt file. The line number should match with the indices of cells in cell2ind.txt file.
    ```
    0,0,1,0,0,0..
    0,0,0,0,1,0..
    0,0,0,0,0,0..
    ```
    <br/>
- `cell2cndeletion.txt`:
    A comma-delimited file where each row has 718 binary values indicating copy number deletion (1) (0 for no copy number deletion).
    ```
    0,0,0,0,0,0..
    0,1,0,0,0,0..
    0,0,0,0,1,0..
    ```
    <br/>
- `cell2amplification.txt`:
    A comma-delimited file where each row has 718 binary values indicating copy number amplification (1) (0 for no copy number amplification).
    ```    
    0,0,0,0,0,0..
    0,0,0,1,0,0..
    0,1,0,0,0,0..
    ```

### Input Data for Training

Drug response data were obtained by harmonizing the Cancer Therapeutics Response Portal (CTRP) and Genomics of Drug Sensitivity in Cancer (GDSC) databases.([Park et al. Nat Cancer (2024)](https://doi.org/10.1038/s43018-024-00740-1))

The data from the two datasets were harmonized as follows. Drug information: each molecule’s published name, synonym or SMILES (Simplified Molecular Input Line Entry System) string was queried using PubChemPy. The associated InChIKey was extracted and used to identify duplicate drugs (within or between datasets). Cell viability data: for CTRP, the vehicle control-normalized average percent viability files were used.([Park et al. Nat Cancer (2024)](https://doi.org/10.1038/s43018-024-00740-1))


- `training_data.txt`:
    A tab-delimited file containing all data points that you want to use to train the model. The 1st column is identification of cells (genotypes), the 2nd column is a SMILES string of the drug and the 3rd column is an observed drug response in a floating point number, and the 4th column is source where the data was obtained from.
    ```
    HS633T_SOFT_TISSUE              CC1=C(C(=O)N(C2=NC(=NC=C12)NC3=NC=C(C=C3)N4CCNCC4)C5CCCC5)C(=O)C     0.67   GDSC2
    KINGS1_CENTRAL_NERVOUS_SYSTEM   CC1=C(C(=O)N(C2=NC(=NC=C12)NC3=NC=C(C=C3)N4CCNCC4)C5CCCC5)C(=O)C     0.64   GDSC1
    ```
    <br/>

- `hierarchy.cx2`:
    Hierarchy in HCX format used to create a visible neural network.

<img src="../docs/images/nest_vnn.png" alt="VNN Training" style="width:70%;">

### Training command

In [2]:
!cellmaps_vnncmd.py train ./6.cellmaps_vnn --inputdir ../examples --gene2id ../examples/gene2ind.txt --cell2id ../examples/cell2ind.txt --training_data ../examples/training_data.txt --mutations ../examples/cell2mutation.txt --cn_deletions ../examples/cell2cndeletion.txt --cn_amplifications ../examples/cell2cnamplification.txt --genotype_hiddens 4 --lr 0.0005 --epoch 30 --batchsize 64 --optimize 1 --zscore_method auc

For required and optional arguments refer to [documentation](https://cellmaps-vnn.readthedocs.io/en/latest/usage_command_line.html#training-mode-and-prediction-and-interpretation-mode).

### Training Outputs

- `model_final.pt`:
    The trained model.

- `std.txt`:
    Standard deviation values for a given training data based on the specified z-score method (‘zscore’ and ‘robustz’).
  
    ```
    GDSC1   0.0     1.0
    GDSC2   0.0     1.0
    ```

### Input Data for Prediction

- `test_data.txt`:
    A tab-delimited file containing all data points that you want to estimate drug response for. The 1st column is identification of cells (genotypes), the 2nd column is a SMILES string of the drug and the 3rd column is an observed drug response in a floating point number, and the 4th column is source where the data was obtained from.
  
    ```
    EW24_BONE       CC1=C(C(=O)N(C2=NC(=NC=C12)NC3=NC=C(C=C3)N4CCNCC4)C5CCCC5)C(=O)C    0.99    GDSC1
    ES7_BONE	    CC1=C(C(=O)N(C2=NC(=NC=C12)NC3=NC=C(C=C3)N4CCNCC4)C5CCCC5)C(=O)C	0.65	GDSC2
    ```
    <br/>
- `model_final.pt`:
    The trained model.

  <img src="../docs/images/nestvnn_pred_int.png" alt="VNN Drug Response Prediction" style="width:70%;">

### Prediction and Interpretation command

In [3]:
!cellmaps_vnncmd.py predict ./7.cellmaps_vnn_prediction --inputdir ./6.cellmaps_vnn --gene2id ../examples/gene2ind.txt --cell2id ../examples/cell2ind.txt --predict_data ../examples/test_data.txt --mutations ../examples/cell2mutation.txt --cn_deletions ../examples/cell2cndeletion.txt --cn_amplifications ../examples/cell2cnamplification.txt --batchsize 64 --zscore_method auc

Starting score calculation
Time taken to load features: 1.1753
/Users/jlenkiewicz/anaconda3/envs/cellmaps_env/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
Drug 1 completed in 0.8940 seconds
FAIRSCAPE hidden files registration:   0%|     | 5/1698 [00:01<05:46,  4.88it/s]


### Prediction Outputs

- `predict.txt`:
    File containing prediction results.
  
    ```
    8.3823e-01
    7.6122e-01
    7.3318e-01
    ```
    </br>

- `predict_feature_grad_0.txt` (`predict_feature_grad_1.txt`, `predict_feature_grad_2.txt`):
    Files containing the gradients for each feature.

    ```
    -3.5083e-04     -1.5043e-03     1.4109e-02      4.2064e-04      -2.6275e-02 ..
    7.7715e-03      -2.1520e-02     -5.5406e-03     -1.3177e-03     -4.9724e-05 ..
    9.3111e-03      -2.0868e-02     3.6902e-05      6.6282e-03      -3.1622e-03 ..
    ```
    <br/>
    
- `hidden directory`:
    Directory with files containing the gradients of the hidden layer outputs.

- `rlipp.out`:
    Output file with interpretation of predictions made by VNN.

    ```
    Term    P_rho   P_pval  C_rho   C_pval  RLIPP
    0       1.000e+00       0.000e+00       9.951e-01       0.000e+00       1.005e+00
    1       7.716e-01       5.312e-67       3.923e-01       1.064e-13       1.967e+00
    2       5.519e-01       6.151e-28       4.664e-01       2.182e-19       1.183e+00
    3       7.867e-01       2.638e-71       7.438e-01       7.026e-60       1.058e+00
    ```
    <br/>

- ``gene_rho.out``:
    Output file with Spearman correlation between gene embeddings and predicted AUC.

    ```
    Gene    Rho     P_val
    ABCB1   -1.215e-01      2.667e-02
    ABCC3   -9.125e-03      8.682e-01
    ABL1    5.741e-02       2.962e-01
    ABL2    -5.068e-02      3.565e-01
    ```

### Input Data for Annotation

- `rlipp.out`:
    File with interpretation scores of the predictions made by VNN model. Disease column is optional.
  
    ```
    Term    P_rho   P_pval  C_rho   C_pval  RLIPP   Disease
    NEST    9.99800e-01     0.00000e+00     9.33000e-01     4.10702e-147    1.07150e+00     Leukemia
    NEST:6  7.71750e-01     7.47000e-64     7.58600e-01     1.36101e-61     1.01750e+00     Leukemia
    NEST:58 6.44850e-01     1.44552e-38     6.62900e-01     1.62600e-40     9.73000e-01     Leukemia
    ```
    <br/>
- ``hierarchy.cx2``:
    Hierarchy in HCX format that will be annotated with interpretation results that will help determine importance of the subsystems in the hierarchical network.

  <img src="../docs/images/nestvnn_annot.png" alt="VNN Hierarchy Annotation" style="width:60%;">  

### Annotation command

In [4]:
!cellmaps_vnncmd.py annotate ./8.cellmaps_vnn_annotation --model_predictions ./7.cellmaps_vnn_prediction

### Annotation Outputs

- `hierarchy.cx2`:
    File with hierarchy in HCX format annotated with interpretation results that will help determine importance of the subsystems in the hierarchical network.

- `rlipp.out`:
    Aggregated interpretation scores of each provided RO-crates with prediction and interpretation results.
    
    ```
    Term    P_rho   P_pval  C_rho   C_pval  RLIPP   Disease
    0       1.00000e+00     0.00000e+00     9.95100e-01     0.00000e+00     1.00500e+00     unspecified
    1       7.71600e-01     5.31200e-67     3.92300e-01     1.06400e-13     1.96700e+00     unspecified
    2       5.51900e-01     6.15100e-28     4.66400e-01     2.18200e-19     1.18300e+00     unspecified
    ```